<a href="https://colab.research.google.com/github/ccastelblancob/Metodos_Comptacion_Estadistica/blob/Tareas/computacion_tarea_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Modelo LDA

In [ ]:
# Solo se va a lematizar.
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text): #  gensim.utils.simple_preprocess tokeniza el texto
        # Se eliminan las palabras vacias.
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) >3 or token == "no":
            result.append(lemmatize_stemming(token))

    return result

In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/IA-2023-2/tweets.csv')
data_text = data[["tweet"]]
data_text["index"] = data_text.index
documents = data_text
len(documents)

<ipython-input-4-0bac8b4183cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text["index"] = data_text.index


3063

In [ ]:
doc_sample = documents[documents['index'] == 3025].values[0][0]
print('documento original: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n documento tokenizado y lematizado: ')
print(preprocess(doc_sample))

documento original: 
['', 'no', 'me', 'siento', 'bien', 'pero', 'me', 'gusta', 'en', 'el', 'buen', 'sentido', 'si', 'eso', 'tiene', 'sentido']


 documento tokenizado y lematizado: 
['no', 'siento', 'bien', 'pero', 'gusta', 'buen', 'sentido', 'tiene', 'sentido']


In [ ]:
#la función map toma cada fila de documents['tweet'] y la pasa por la función preprocess
processed_docs = documents['tweet'].map(preprocess)
processed_docs[:10]

0    [verdadera, razón, estás, triste, apegas, pers...
1           [mayor, problema, pensar, demasiado, todo]
2        [peor, tristeza, tristeza, enseñes, esconder]
3    [no, puedo, hacerte, entender, no, puedo, hace...
4    [no, creo, nadie, realmente, entienda, cansado...
5    [peor, sensación, cuando, algo, mata, dentro, ...
6    [cuando, estoy, herido, cierro, convierto, per...
7    [pensar, demasiado, arruinar, arruinar, situac...
8    [estoy, decepcionado, mismo, porque, mejor, el...
9    [estoy, cansada, esto, cuerpo, está, cansada, ...
Name: tweet, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


0 apegas
1 atención
2 cuando
3 dejaras
4 demasiado
5 distanciado
6 están
7 estás
8 gente
9 haces
10 ignoran


In [ ]:
#no_below: especifica el número mínimo de documentos en los que debe aparecer una palabra para que se mantenga en el diccionario
#no_above: especifica la proporción máxima de documentos en los que una palabra puede aparecer para que se mantenga en el diccionario.
#keep_n: controla el número total máximo de palabras que se mantendrán en el diccionario después del proceso de filtrado.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
# Se crea una bolsa de palabras
# Primera enntrada: Identificador único de la palabra. Segunda entrada: Frecuencia de la palabra.
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[3025]

[(18, 1), (22, 1), (64, 1), (103, 2), (107, 1), (152, 1), (189, 1)]

In [ ]:
bow_doc_15 = bow_corpus[3025]
for i in range(len(bow_doc_15)):
    print("Word {} (\"{}\") aparece {} una vez.".format(bow_doc_15[i][0],
                                               dictionary[bow_doc_15[i][0]],
bow_doc_15[i][1]))

Word 18 ("no") aparece 1 una vez.
Word 22 ("bien") aparece 1 una vez.
Word 64 ("siento") aparece 1 una vez.
Word 103 ("sentido") aparece 2 una vez.
Word 107 ("pero") aparece 1 una vez.
Word 152 ("gusta") aparece 1 una vez.
Word 189 ("tiene") aparece 1 una vez.


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break


[(0, 0.1253782036951276),
 (1, 0.38436155419005563),
 (2, 0.21108741411643359),
 (3, 0.16634826356371224),
 (4, 0.2234113770703681),
 (5, 0.22839805829516904),
 (6, 0.7280941313765652),
 (7, 0.2342823743667716),
 (8, 0.19527570849762166),
 (9, 0.19772871986412627)]


In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.171*"no" + 0.033*"para" + 0.032*"alguien" + 0.026*"puedo" + 0.022*"nunca" + 0.020*"bien" + 0.019*"porque" + 0.015*"solo" + 0.014*"simplemente" + 0.014*"todo"
Topic: 1 
Words: 0.075*"no" + 0.044*"pero" + 0.037*"como" + 0.032*"solo" + 0.030*"para" + 0.026*"vida" + 0.019*"cuando" + 0.018*"porque" + 0.018*"siempre" + 0.018*"personas"
Topic: 2 
Words: 0.062*"no" + 0.047*"quiero" + 0.042*"todo" + 0.034*"estar" + 0.031*"solo" + 0.026*"para" + 0.023*"mierda" + 0.023*"cuando" + 0.022*"cansado" + 0.020*"puedo"
Topic: 3 
Words: 0.055*"no" + 0.044*"cuando" + 0.038*"todos" + 0.036*"siento" + 0.034*"estoy" + 0.031*"pero" + 0.023*"todo" + 0.021*"como" + 0.020*"siempre" + 0.019*"gente"
Topic: 4 
Words: 0.084*"no" + 0.049*"nadie" + 0.036*"para" + 0.034*"solo" + 0.026*"pero" + 0.022*"alguien" + 0.021*"cuando" + 0.021*"estoy" + 0.018*"mismo" + 0.017*"persona"


In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.069*"no" + 0.044*"todos" + 0.027*"pero" + 0.026*"mierda" + 0.025*"estar" + 0.024*"como" + 0.021*"solo" + 0.021*"alguien" + 0.020*"seré" + 0.020*"bien"
Topic: 1 Word: 0.034*"todo" + 0.031*"no" + 0.029*"para" + 0.027*"cuando" + 0.024*"como" + 0.023*"importa" + 0.022*"siempre" + 0.019*"hacer" + 0.018*"quiero" + 0.016*"porque"
Topic: 2 Word: 0.031*"para" + 0.027*"no" + 0.024*"tengo" + 0.022*"nada" + 0.018*"nunca" + 0.017*"cada" + 0.016*"pensamientos" + 0.016*"amigos" + 0.016*"también" + 0.016*"realmente"
Topic: 3 Word: 0.034*"vida" + 0.030*"no" + 0.028*"quiero" + 0.020*"nadie" + 0.020*"vivir" + 0.018*"solo" + 0.016*"estaré" + 0.016*"siempre" + 0.015*"mejor" + 0.015*"alguien"
Topic: 4 Word: 0.047*"estoy" + 0.041*"solo" + 0.037*"no" + 0.023*"mismo" + 0.023*"puedo" + 0.020*"cosas" + 0.020*"dolor" + 0.018*"personas" + 0.018*"esto" + 0.018*"cuando"


In [ ]:
processed_docs[3025]

['no',
 'siento',
 'bien',
 'pero',
 'gusta',
 'buen',
 'sentido',
 'tiene',
 'sentido']

In [ ]:
for index, score in sorted(lda_model[bow_corpus[3025]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.908891499042511	 
Topic: 0.084*"no" + 0.049*"nadie" + 0.036*"para" + 0.034*"solo" + 0.026*"pero" + 0.022*"alguien" + 0.021*"cuando" + 0.021*"estoy" + 0.018*"mismo" + 0.017*"persona"

Score: 0.022902699187397957	 
Topic: 0.055*"no" + 0.044*"cuando" + 0.038*"todos" + 0.036*"siento" + 0.034*"estoy" + 0.031*"pero" + 0.023*"todo" + 0.021*"como" + 0.020*"siempre" + 0.019*"gente"

Score: 0.022746924310922623	 
Topic: 0.171*"no" + 0.033*"para" + 0.032*"alguien" + 0.026*"puedo" + 0.022*"nunca" + 0.020*"bien" + 0.019*"porque" + 0.015*"solo" + 0.014*"simplemente" + 0.014*"todo"

Score: 0.02273695170879364	 
Topic: 0.062*"no" + 0.047*"quiero" + 0.042*"todo" + 0.034*"estar" + 0.031*"solo" + 0.026*"para" + 0.023*"mierda" + 0.023*"cuando" + 0.022*"cansado" + 0.020*"puedo"

Score: 0.022721877321600914	 
Topic: 0.075*"no" + 0.044*"pero" + 0.037*"como" + 0.032*"solo" + 0.030*"para" + 0.026*"vida" + 0.019*"cuando" + 0.018*"porque" + 0.018*"siempre" + 0.018*"personas"


In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[3025]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6417878866195679	 Topic: 0.069*"no" + 0.044*"todos" + 0.027*"pero" + 0.026*"mierda" + 0.025*"estar" + 0.024*"como" + 0.021*"solo" + 0.021*"alguien" + 0.020*"seré" + 0.020*"bien"

Score: 0.2897302806377411	 Topic: 0.034*"todo" + 0.031*"no" + 0.029*"para" + 0.027*"cuando" + 0.024*"como" + 0.023*"importa" + 0.022*"siempre" + 0.019*"hacer" + 0.018*"quiero" + 0.016*"porque"

Score: 0.02315969206392765	 Topic: 0.034*"vida" + 0.030*"no" + 0.028*"quiero" + 0.020*"nadie" + 0.020*"vivir" + 0.018*"solo" + 0.016*"estaré" + 0.016*"siempre" + 0.015*"mejor" + 0.015*"alguien"

Score: 0.022766081616282463	 Topic: 0.031*"para" + 0.027*"no" + 0.024*"tengo" + 0.022*"nada" + 0.018*"nunca" + 0.017*"cada" + 0.016*"pensamientos" + 0.016*"amigos" + 0.016*"también" + 0.016*"realmente"

Score: 0.022556038573384285	 Topic: 0.047*"estoy" + 0.041*"solo" + 0.037*"no" + 0.023*"mismo" + 0.023*"puedo" + 0.020*"cosas" + 0.020*"dolor" + 0.018*"personas" + 0.018*"esto" + 0.018*"cuando"


In [ ]:
unseen_document = 'No entiendo que hace esto, ayuda'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.7950260639190674	 Topic: 0.084*"no" + 0.049*"nadie" + 0.036*"para" + 0.034*"solo" + 0.026*"pero"
Score: 0.05227319151163101	 Topic: 0.171*"no" + 0.033*"para" + 0.032*"alguien" + 0.026*"puedo" + 0.022*"nunca"
Score: 0.05127741023898125	 Topic: 0.075*"no" + 0.044*"pero" + 0.037*"como" + 0.032*"solo" + 0.030*"para"
Score: 0.05076996982097626	 Topic: 0.055*"no" + 0.044*"cuando" + 0.038*"todos" + 0.036*"siento" + 0.034*"estoy"
Score: 0.05065333470702171	 Topic: 0.062*"no" + 0.047*"quiero" + 0.042*"todo" + 0.034*"estar" + 0.031*"solo"


In [ ]:
unseen_document = 'No entiendo que hace esto, ayuda'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.7969995737075806	 Topic: 0.084*"no" + 0.049*"nadie" + 0.036*"para" + 0.034*"solo" + 0.026*"pero"
Score: 0.051063649356365204	 Topic: 0.171*"no" + 0.033*"para" + 0.032*"alguien" + 0.026*"puedo" + 0.022*"nunca"
Score: 0.050672899931669235	 Topic: 0.075*"no" + 0.044*"pero" + 0.037*"como" + 0.032*"solo" + 0.030*"para"
Score: 0.050639934837818146	 Topic: 0.062*"no" + 0.047*"quiero" + 0.042*"todo" + 0.034*"estar" + 0.031*"solo"
Score: 0.050623971968889236	 Topic: 0.055*"no" + 0.044*"cuando" + 0.038*"todos" + 0.036*"siento" + 0.034*"estoy"


Modelo SVD

In [9]:
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
#nltk.download('stopwords')
#nltk.download('punkt')

In [6]:
def initial_clean(text):
    """
    Función para limpiar texto de sitios web, direcciones de correo electrónico y cualquier puntuación.
    """
    text = text.lower() # lower case the text
    text = nltk.word_tokenize(text)
    return text

stopwords_corpus = nltk.corpus.stopwords
spa_stop_words = stopwords_corpus.words('spanish')
stopset = spa_stop_words + list(string.punctuation)
def remove_stop_words(text):
    """
    Función que remueve las palabras vacías del texto
    """
    return [word for word in text if word not in stopset]

def apply_all(text):
    """
    Función para aplicar todas las funciones anteriores
    """
    words = remove_stop_words(initial_clean(text))
    return " ".join([i for i in words])

In [10]:
data["processed_docs_1"] = data["tweet"].apply(lambda x: apply_all(x))
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, max_features=100)
tfidf_matrix = tfidf_vectorizer.fit_transform(data["processed_docs_1"])

In [11]:
# Realizar la descomposición SVD
num_topics = 10  # Número de tópicos a extraer
svd = TruncatedSVD(n_components=num_topics)
svd_matrix = svd.fit_transform(tfidf_matrix)

In [12]:
terms = tfidf_vectorizer.get_feature_names_out()
for i, topic in enumerate(svd.components_):
    top_terms_idx = topic.argsort()[::-1][:10]  # Mostrar las 10 palabras principales por tópico
    top_terms = [terms[idx] for idx in top_terms_idx]
    print(f'Tópico {i + 1}: {" ".join(top_terms)}')

Tópico 1: solo quiero si ser bien alguien siento tan nadie siempre
Tópico 2: solo quiero morir aquí realidad vivo oh amigos dormir sentir
Tópico 3: ser tan puedo siento quiero nunca feliz persona vida hacer
Tópico 4: bien ser veces hacer puedo feliz cosas decir momento sonrisa
Tópico 5: nadie importa bien siempre realmente nunca alguien vez cuenta puede
Tópico 6: siento tan puedo siempre gente mierda vez cansado hacer triste
Tópico 7: quiero siempre siento vida simplemente hacer puedo aquí tan importa
Tópico 8: vida siempre alguien nunca odio personas gente noche solo demasiado
Tópico 9: vida puedo nadie mierda hacer bien odio día importa siento
Tópico 10: alguien nunca tan realmente importa siento quiero vez cómo hablar
